In [1]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import math
def NumbertoTime(sec):
 TOS=str(math.floor(sec%100)).zfill(2)
 TTime=sec/100
 TS=str(math.floor(TTime%60)).zfill(2)
 TTime=TTime/60
 TM=str(math.floor(TTime%60)).zfill(2)
 TTime=TTime/60
 TH=str(math.floor(TTime%60)).zfill(2)
 return TH+TM+TS
def TimetoNumber(time):
    time=time.zfill(8)
    sec=int(time[:2])*360000+int(time[2:4])*6000+int(time[4:6])*100+int(time[6:8])
    return sec
#I020 = [ line.strip('\n').split(",") for line in open('/Users/kevin/Downloads/TickData/Futures_20170815_I020.csv')][1:]

# 51 取得即時報價資訊

In [2]:
#function
# -*- coding: UTF-8 -*-
#載入相關套件
import time
import datetime
import tailer

#取得當天日期
Date=time.strftime("%Y%m%d")
#設定檔案位置
DataPath="/Users/kevin/Google 雲端硬碟/碩班/ML/"

#開啟這三個檔案
MatchFile=open(DataPath+Date+'_Match.txt')
OrderFile=open(DataPath+Date+'_Commission.txt')
UpDn5File=open(DataPath+Date+'_UpDn5.txt')

#持續取得成交資訊
def getMatch():
 return tailer.follow(MatchFile,0)

#持續取得委託資訊
def getOrder():
 return tailer.follow(OrderFile,0)

#持續取得上下五檔價資訊
def getUpDn5():
 return tailer.follow(UpDn5File,0)

#取得最新一筆成交資訊
def getLastMatch():
 return tailer.tail(MatchFile,3)[-2].split(",")

#取得最新一筆委託資訊
def getLastOrder():
 return tailer.tail(OrderFile,3)[-2].split(",")

#取得最新一筆上下五檔價資訊
def getLastUpDn5():
 return tailer.tail(UpDn5File,3)[-2].split(",")



FileNotFoundError: [Errno 2] No such file or directory: 'D:/data/20180710_Match.txt'

# 52 計算每分鐘開高收低價

In [4]:
#52
# -*- coding: UTF-8 -*-

#取得報價資訊，詳情請查看技巧51
exec(open("/Users/kevin/Downloads/Strategy_Sample_Python/即時範例/function.py").read())


#定義指標變數
OHLC=[]

#取得成交資訊
for i in getMatch():		
  MatchInfo=i.split(',')
  #定義HHMM的時間字串，方便進行分鐘轉換判斷
  HMTime=MatchInfo[0][0:2]+MatchInfo[0][3:5]
  MatchPrice=int(MatchInfo[1])
  #若OHLC為空，先填值
  if len(OHLC)==0:
   OHLC.append([HMTime,MatchPrice,MatchPrice,MatchPrice,MatchPrice])	
  else:
   #進行該分鐘是否結束	
   if HMTime==OHLC[-1][0]:
    #進行高、低價判斷
    if MatchPrice>OHLC[-1][2]:
     OHLC[-1][2]=MatchPrice
    if MatchPrice<OHLC[-1][3]:
     OHLC[-1][3]=MatchPrice
    OHLC[-1][4]=MatchPrice
   else:
   #該分鐘結束則加入新行
    OHLC.append([HMTime,MatchPrice,MatchPrice,MatchPrice,MatchPrice])
  #顯示當前開高低收
  print(OHLC[-1])

KeyboardInterrupt: 

# 53 計算每分鐘累積量

In [ ]:
#53
# -*- coding: UTF-8 -*-

#取得報價資訊，詳情請查看技巧51
exec(open("/Users/kevin/Downloads/Strategy_Sample_Python/即時範例/function.py").read())

#定義指標變數
Qty=[]
lastAmount=0

#取得成交資訊
for i in getMatch():		
  MatchInfo=i.split(',')
  #定義HHMM的時間字串，方便進行分鐘轉換判斷
  HMTime=MatchInfo[0][0:2]+MatchInfo[0][3:5]
  MatchAmount=int(MatchInfo[3])
  #進行每分鐘價格計算
  if len(Qty)==0:
   Qty.append([HMTime,0])
   lastAmount=MatchAmount
  else:
   if HMTime==Qty[-1][0]:
    Qty[-1][1]=MatchAmount-lastAmount
   else:
    Qty.append([HMTime,0])
    lastAmount=MatchAmount
 
  print (Qty)

# 54 計算買賣方每筆平均成交口數

In [ ]:
#54
# -*- coding: UTF-8 -*-

#取得報價資訊，詳情請查看技巧51
#execfile('function.py')

#取得成交資訊
for i in getMatch():		
  MatchInfo=i.split(',')
  MatchTime=MatchInfo[0]
  MatchAmount=int(MatchInfo[3])
  MatchBCnt=int(MatchInfo[4])
  MatchSCnt=int(MatchInfo[5])

  #進行平均買賣口計算
  avgB=float(MatchAmount)/MatchBCnt
  avgS=float(MatchAmount)/MatchSCnt
  
  print (MatchTime,"avgB",avgB,"avgS",avgS)

# 56 計算內外盤總量

In [ ]:
#56-1
# -*- coding: UTF-8 -*-

#取得報價資訊，詳情請查看技巧51
#execfile('function.py')

#定義指標變數
OutDesk=0
InDesk=0

#取得成交資訊
for i in getMatch():		
  MatchInfo=i.split(',')
  MatchTime=MatchInfo[0]
  MatchPrcie=int(MatchInfo[1])
  MatchQty=int(MatchInfo[2])
  #取得上下五檔價資訊
  UpDn5Info=getLastUpDn5()
  Dn1Price=int(UpDn5Info[1])
  Up1Price=int(UpDn5Info[11])

  #進行內外盤判斷
  if MatchPrcie>=Up1Price:
   OutDesk+=MatchQty
  if MatchPrcie<=Dn1Price:
   InDesk+=MatchQty
   
  print (MatchTime,"OutDesk",OutDesk,"InDesk",InDesk)

In [ ]:
#56-2
# -*- coding: UTF-8 -*-

#取得報價資訊，詳情請查看技巧51
#execfile('function.py')

#定義指標變數
OutDesk=0
InDesk=0

#取得成交資訊
for i in getMatch():		
  MatchInfo=i.split(',')
  MatchTime=MatchInfo[0]
  MatchPrcie=int(MatchInfo[1])
  #取得上下五檔價資訊
  UpDn5Info=getLastUpDn5()
  Dn1Price=int(UpDn5Info[1])
  Up1Price=int(UpDn5Info[11])
  
  #進行內外盤判斷
  if MatchPrcie>=Up1Price:
   OutDesk+=1
  if MatchPrcie<=Dn1Price:
   InDesk+=1

  print (MatchTime,"OutDesk",OutDesk,"InDesk",InDesk)


# 57計算內外盤比率

In [ ]:
#57
# -*- coding: UTF-8 -*-

#取得報價資訊，詳情請查看技巧51
#execfile('function.py')

#定義指標變數
OutDesk=0
InDesk=0

#取得成交資訊
for i in getMatch():		
  MatchInfo=i.split(',')
  MatchTime=MatchInfo[0]
  MatchPrcie=int(MatchInfo[1])
  MatchQty=int(MatchInfo[2])
  #取得上下五檔價資訊
  UpDn5Info=getLastUpDn5()
  Dn1Price=int(UpDn5Info[1])
  Up1Price=int(UpDn5Info[11])

  #進行內外盤判斷
  if MatchPrcie>=Up1Price:
   OutDesk+=MatchQty
  if MatchPrcie<=Dn1Price:
   InDesk+=MatchQty

  #內外盤比率計算
  OutInRatio=float(OutDesk)/(OutDesk+InDesk)

  print (MatchTime,"OutDesk Ratio",OutInRatio)

# 58 計算買賣方委託總量

In [ ]:
#58
# -*- coding: UTF-8 -*-

#取得報價資訊，詳情請查看技巧51
#execfile('function.py')

#取得委託資訊
for i in getOrder():		
  OrderInfo=i.split(',')
  OrderTime=OrderInfo[0]
  OrderBAmount=int(OrderInfo[2])
  OrderSAmount=int(OrderInfo[4])

  #委託總量相減，並顯示
  print (OrderTime,"diffOrder",OrderBAmount-OrderSAmount)

# 59 計算買賣方委託平均量

In [ ]:
#59
# -*- coding: UTF-8 -*-

#取得報價資訊，詳情請查看技巧51
#execfile('function.py')

#取得委託資訊
for i in getOrder():		
  OrderInfo=i.split(',')
  OrderTime=OrderInfo[0]
  OrderBCnt=int(OrderInfo[1])
  OrderBAmount=int(OrderInfo[2])
  OrderSCnt=int(OrderInfo[3])
  OrderSAmount=int(OrderInfo[4])

  #委託平均口數計算，並顯示
  print (OrderTime,"avgOrderB",float(OrderBAmount)/OrderBCnt,"avgOrderS",float(OrderSAmount)/OrderSCnt)


# 60 計算動態委託量變化

In [ ]:
#60
# -*- coding: UTF-8 -*-

#取得報價資訊，詳情請查看技巧51
#execfile('function.py')

#定義指標變數
lastOrderBAmount=0
lastOrderSAmount=0

#取得委託資訊
for i in getOrder():		
  OrderInfo=i.split(',')
  OrderTime=OrderInfo[0]
  OrderBAmount=int(OrderInfo[2])
  OrderSAmount=int(OrderInfo[4])

  #紀錄上一筆總量資訊
  if lastOrderBAmount==0 and lastOrderSAmount==0:
   lastOrderBAmount=OrderBAmount
   lastOrderSAmount=OrderSAmount
   continue
  else:
   #進行計算差值	
   diffOrderBAmount=OrderBAmount-lastOrderBAmount
   diffOrderSAmount=OrderSAmount-lastOrderSAmount
   lastOrderBAmount=OrderBAmount
   lastOrderSAmount=OrderSAmount

  print (OrderTime,"diffOrderBAmount",diffOrderBAmount,"diffOrderSAmount",diffOrderSAmount)

# 61 計算上下五檔平均成本

In [ ]:
#61
# -*- coding: UTF-8 -*-

#取得報價資訊，詳情請查看技巧51
#execfile('function.py')

#取得上下五檔價量資訊
for i in getUpDn5():		
 UpDn5Info=i.split(',')
 UpDn5Time=UpDn5Info[0]
 totalUpPrice=0
 totalUpQty=0
 totalDnPrice=0
 totalDnQty=0
 
 #開始進行上下五檔加權平均值
 for j in range(0,5):
  totalDnPrice+=int(UpDn5Info[1+2*j])*int(UpDn5Info[2+2*j])
  totalDnQty+=int(UpDn5Info[2+2*j])
  totalUpPrice+=int(UpDn5Info[11+2*j])*int(UpDn5Info[12+2*j])
  totalUpQty+=int(UpDn5Info[12+2*j])
 
 print (UpDn5Time,"avgUpPrice",float(totalUpPrice)/totalUpQty,"avgDnPrice",float(totalDnPrice)/totalDnQty  )

# 62 計算價格MA指標

In [ ]:
#62
# -*- coding: UTF-8 -*-

#取得報價資訊，詳情請查看技巧51
#execfile('function.py')

#定義指標變數
MAarray=[]
MAnum=10
lastHMTime=""

#取得成交資訊
for i in getMatch():		
 MatchInfo=i.split(',')
 #定義HHMM的時間字串，方便進行分鐘轉換判斷
 HMTime=MatchInfo[0][0:2]+MatchInfo[0][3:5]
 MatchPrice=int(MatchInfo[1])

 #進行MA的計算
 if len(MAarray)==0:
  MAarray+=[MatchPrice]
  lastHMTime=HMTime
 else:
  if HMTime==lastHMTime:
   MAarray[-1]=MatchPrice
  elif HMTime!=lastHMTime:
   if len(MAarray)<MAnum:
    MAarray+=[MatchPrice]
   elif len(MAarray)==MAnum:
   	MAarray=MAarray[1:]+[MatchPrice]
   lastHMTime=HMTime

 print (HMTime,"MAvalue",float(sum(MAarray))/len(MAarray))

# 63 計算量MA指標

In [ ]:
#63
# -*- coding: UTF-8 -*-

#取得報價資訊，詳情請查看技巧51
#execfile('function.py')

#定義指標變數
Qty=[]
QMA=0
MAnum=5
lastHMTime=""
lastAmount=0

#取得成交資訊
for i in getMatch():		
  MatchInfo=i.split(',')
  #定義HHMM的時間字串，方便進行分鐘轉換判斷
  HMTime=MatchInfo[0][0:2]+MatchInfo[0][3:5]
  MatchAmount=int(MatchInfo[3])
  
  #進行量MA的計算
  if len(Qty)==0:
   Qty+=[0]
   lastHMTime=HMTime
   lastAmount=MatchAmount
  else:
   if HMTime==lastHMTime:
    Qty[-1]=MatchAmount-lastAmount
   else:
    if len(Qty)==MAnum:
     QMA=sum(Qty)/len(Qty)
     print QMA
     Qty=Qty[1:]+[0]
    else: 
     Qty+=[0]
    lastHMTime=HMTime
    lastAmount=MatchAmount
  #顯示量MA
  print (Qty)

# 64 計算每分鐘價格變化趨勢

In [ ]:
#64
# -*- coding: UTF-8 -*-

#取得報價資訊，詳情請查看技巧51
#execfile('function.py')

#定義指標變數
closePrice=[]
lastHMTime=""

#取得成交資訊
for i in getMatch():		
 MatchInfo=i.split(',')
 #定義HHMM的時間字串，方便進行分鐘轉換判斷
 HMTime=MatchInfo[0][0:2]+MatchInfo[0][3:5]
 MatchPrice=int(MatchInfo[1])
 
 #進行每分鐘收盤價計算
 if len(closePrice)==0:
  closePrice+=[MatchPrice]
  lastHMTime=HMTime
 else:
  if HMTime==lastHMTime:
   closePrice[-1]= MatchPrice
  elif HMTime!=lastHMTime:
   closePrice+=[MatchPrice]
   lastHMTime=HMTime
 
 #顯示當前價
 print ("current Price:",closePrice[-1])

# 65 計算固定Tick數開高低收價

In [ ]:
#65
# -*- coding: UTF-8 -*-

#取得報價資訊，詳情請查看技巧51
#execfile('function.py')

#定義指標變數
TickMA200=[]
TickOHLC=[]

#取得成交資訊
for i in getMatch():		
 MatchInfo=i.split(',')
 MatchTime=MatchInfo[0]
 MatchPrice=int(MatchInfo[1])
 
 #將Tick相加
 TickMA200+=[MatchPrice]
 
 #當tick200筆時，進行開高低收統計
 if len(TickMA200)==200:
  TickOHLC+=[[MatchTime,TickMA200[0],max(TickMA200),min(TickMA200),TickMA200[-1]]]
  TickMA200=[]
  print (TickOHLC[-1])

# 66 計算大戶指標

In [ ]:
#66
# -*- coding: UTF-8 -*-

#取得報價資訊，詳情請查看技巧51
#execfile('function.py')

#定義指標變數
lastBcnt=0
lastScnt=0
accB=0
accS=0

#取得成交資訊
for i in getMatch():		
 MatchInfo=i.split(',')
 MatchTime=MatchInfo[0]
 MatchPrice=int(MatchInfo[1])
 MatchQty=int(MatchInfo[2])
 MatchBcnt=int(MatchInfo[4])
 MatchScnt=int(MatchInfo[5])
 
 #儲存上一筆最新總筆數 
 if lastBcnt==0 and lastScnt==0:
  lastBcnt=MatchBcnt
  lastScnt=MatchScnt
 else:
  #計算相差筆數	
  diffBcnt=MatchBcnt-lastBcnt
  diffScnt=MatchScnt-lastScnt
  #進行數量判斷
  if MatchQty>=10:
   #進行買賣方判斷
   if diffBcnt==1 and diffScnt>1:
    accB+=MatchQty
    print (MatchTime,MatchPrice,MatchQty,0,accB,accS)
   elif diffScnt==1 and diffBcnt>1:
    accS+=MatchQty
    print (MatchTime,MatchPrice,0,MatchQty,accB,accS)

 lastBcnt=MatchBcnt
 lastScnt=MatchScnt